In [326]:
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint
import pandas as pd
import re

In [327]:
def get_parts_salary_info(salary_info):    
    
    min_salary, max_salary, currency = None, None, None
    
    if salary_info != None and salary_info.text != 'По договорённости':
        salary_info = salary_info.text
        currency = salary_info.split()[-1].replace('.', '')
                
        if 'от' in salary_info:
            min_salary, max_salary = int(''.join(re.findall(r'(\d+)', salary_info))), None
        elif 'до' in salary_info:
            min_salary, max_salary = None, int(''.join(re.findall(r'(\d+)', salary_info)))
        elif '-' in salary_info:
            min_salary, max_salary = int(''.join(salary_info.partition('-')[0].split())), \
                                     int(''.join(re.findall(r'(\d+)', salary_info.partition('-')[2])))
        else:
            min_salary = int(''.join(re.findall(r'(\d+)', salary_info)))
            max_salary = min_salary
            
    return(min_salary, max_salary, currency)


position_name = '+'.join('data scientist'.split())

header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36',
    'Accept': '*/*'}

### hh

In [328]:
main_link = 'http://hh.ru'

response = requests.get(main_link +
    f'/search/vacancy?L_save_area=true&clusters=true&enable_snippets=true&search_field=name&text={position_name}&showClusters=true',
                        headers=header).text
soup = bs(response, 'lxml')

all_position = pd.DataFrame(
    columns=['Наименование вакансии', 'Минимальная зп', 'Максимальная зп', 'Валюта', 'Ссылка на вакансию', 'Сайт-источник'])


page_num = 0

while True:
    position_list = soup.find_all('div', {'class': 'vacancy-serp-item__row vacancy-serp-item__row_header'})
    
    for p in position_list:
        salary_info = p.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
        min_salary, max_salary, currency = get_parts_salary_info(salary_info)
            
        all_position.loc[all_position.shape[0] + 1] = p.find('a', {'data-qa': "vacancy-serp__vacancy-title"}).text, \
                          min_salary, max_salary, currency, \
                          p.find('a', {'data-qa': "vacancy-serp__vacancy-title"})['href'], \
                          'hh'
       
    if not soup.find_all('a', {'class': 'bloko-button HH-Pager-Controls-Next HH-Pager-Control'}):
        break
    
    page_num += 1
    response = requests.get(main_link +
    f'/search/vacancy?L_is_autosearch=false&clusters=true&enable_snippets=true&search_field=name&text={position_name}&page={page_num}',
    headers=header).text
    soup = bs(response, 'lxml')
    
    

print(all_position)



                                 Наименование вакансии Минимальная зп  \
1                                       Data Scientist           None   
2    Специалист по углубленной аналитике данных (Da...           None   
3                                       Data Scientist         120000   
4                                Python Data Scientist         150000   
5                                       Data Scientist         130000   
..                                                 ...            ...   
232                              Стажер data scientist           None   
233                                     Data Scientist           None   
234                              Стажер Data Scientist           None   
235                                     Data Scientist           None   
236  Data Scientist в Корпоративно-Инвестиционный Б...           None   

    Максимальная зп Валюта                                 Ссылка на вакансию  \
1              None   None  https://vologd

### superjob

In [329]:
main_link = 'http://www.russia.superjob.ru'

response = requests.get(main_link +
    f'/vacancy/search/?keywords={position_name}',
                        headers=header).text
soup = bs(response, 'lxml')

page_num = 0

while True:
    
    position_list = soup.find_all('div', {'class': 'iJCa5 f-test-vacancy-item _1fma_ _1JhPh _2gFpt _1znz6 _2nteL'})
    
    for p in position_list:
        salary_info = p.find('span', {'class': '_3mfro _2Wp8I PlM3e _2JVkc _2VHxz'})
        min_salary, max_salary, currency = get_parts_salary_info(salary_info)
      
        all_position.loc[all_position.shape[0] + 1] = p.find('div', {'class': "_3mfro PlM3e _2JVkc _3LJqf"}).text, \
                          min_salary, max_salary, currency, \
                          main_link + p.find('a', {'class': "icMQ_"})['href'], \
                          'superjob' 
    
    page_next = soup.find('a', {'rel': 'next'})
    
    if not page_next:
        break
    
    page_num += 1
    response = requests.get(main_link +
    f'/vacancy/search/?keywords={position_name}&page={page_num}',\
    headers=header).text
    soup = bs(response, 'lxml')

    
print(all_position)


                                 Наименование вакансии Минимальная зп  \
1                                       Data Scientist           None   
2    Специалист по углубленной аналитике данных (Da...           None   
3                                       Data Scientist         120000   
4                                Python Data Scientist         150000   
5                                       Data Scientist         130000   
..                                                 ...            ...   
246                       Программист / Разработчик 1С         130000   
247                       Ведущий специалист по данным           None   
248                    Android Developer (Java/Kotlin)           None   
249                       Исполнительный директор в IT           None   
250                                     Data Scientist           None   

    Максимальная зп Валюта                                 Ссылка на вакансию  \
1              None   None  https://vologd